In [1]:
# %%
import os

os.environ['TF_GPU_THREAD_MODE']='gpu_private'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from datetime import datetime
import sys
import numpy as np
import random
import operator
import matplotlib as plt
import pickle
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.utils import Sequence
import re
import pandas as pd
from pyedflib.highlevel import read_edf, read_edf_header

from sklearn.model_selection import KFold

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
  except RuntimeError as e:
    print(e)

2023-10-27 04:37:38.566326: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-27 04:37:38.566366: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-27 04:37:38.566543: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [34]:
# edf_header.keys() = dict_keys(['technician', 'recording_additional', 'patientname', 'patient_additional', 'patientcode',
# 'equipment', 'admincode', 'sex', 'startdate', 'birthdate', 'gender', 'Duration', 'SignalHeaders', 'channels', 'annotations'])

signal_channel_list = []

patient_name_list = [x for x in os.listdir('./data/CHB/') if re.match('(CHB)|(chb)[0-9]{2,}', x)]
for patient_name in patient_name_list:
    patient_path = os.path.join('./data/CHB/', patient_name)

    edf_list = [x for x in os.listdir(patient_path) if x.endswith('.edf')]

    for edf in edf_list:
        edf_path = os.path.join(patient_path, edf)
        header = read_edf_header(edf_path)
        channels = header['channels']
        if len(channels)>10:
            signal_channel_list.append([edf, channels])


In [38]:
used_channel = signal_channel_list[0][1]
print(used_channel)
print(len(used_channel))
for channel in signal_channel_list[1:]:
    common = [x for x in used_channel if x in channel[1]]
    if len(common)<10:
        continue
    used_channel = common

print(used_channel)
print(len(used_channel))

['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'P7-T7', 'T7-FT9', 'FT9-FT10', 'FT10-T8', 'T8-P8']
23
['FP1-F7', 'F7-T7', 'T7-P7', 'P7-O1', 'FP1-F3', 'F3-C3', 'C3-P3', 'P3-O1', 'FP2-F4', 'F4-C4', 'C4-P4', 'P4-O2', 'FP2-F8', 'F8-T8', 'T8-P8', 'P8-O2', 'FZ-CZ', 'CZ-PZ', 'T8-P8']
19


In [41]:
pd.DataFrame(list(zip(used_channel,range(len(used_channel)))),columns=['label','index']).to_csv('./chb_channel_label.csv', index=False)    

In [26]:
common_channel = signal_channel_list[0][0]
for x in signal_channel_list[1:]:
    if len(common_channel)!=len(common_channel&x[0]):    
        print(len(common_channel&x[0]),x[1])

0 CHB012_27.edf
0 CHB012_28.edf
0 CHB012_29.edf
18 CHB013_04.edf
18 CHB013_05.edf
18 CHB013_06.edf
18 CHB013_07.edf
18 CHB013_08.edf
18 CHB013_09.edf
18 CHB013_10.edf
18 CHB013_11.edf
18 CHB013_12.edf
18 CHB013_13.edf
18 CHB013_14.edf
18 CHB013_15.edf
18 CHB013_16.edf
18 CHB013_18.edf
18 CHB013_24.edf
18 CHB013_30.edf
18 CHB013_36.edf
18 CHB013_37.edf
18 CHB013_38.edf
18 CHB013_39.edf
18 CHB013_40.edf
18 CHB013_47.edf
18 CHB015_01.edf
18 CHB016_18.edf
18 CHB016_19.edf
18 CHB018_01.edf
18 CHB019_01.edf
18 chb17c_13.edf


In [6]:
signal_label = []
patient_name_list = [x for x in os.listdir('./data/CHB/') if re.match('CHB[0-9]{3}', x)]
for patient_name in patient_name_list:
    sample_edf = os.listdir(os.path.join('./data/CHB/',patient_name))[0]
    sample_edf_path = os.path.join('./data/CHB/', patient_name, sample_edf)
    header = read_edf_header('./data/CHB/CHB001/CHB001_01.edf')

    current_signal_label = set()
    for signal_dict in header["SignalHeaders"]:
        current_signal_label.add(signal_dict['label'])
    
    signal_label.append(current_signal_label)
len_signal_label = [len(x) for x in signal_label]
print(len_signal_label)
all_label = set()
for x in signal_label:
    all_label |= x
print(len(all_label))
print(all_label)

[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22]
22
{'T7-FT9', 'P7-O1', 'T8-P8', 'FP1-F3', 'T7-P7', 'FT10-T8', 'P8-O2', 'F3-C3', 'F4-C4', 'F8-T8', 'CZ-PZ', 'FZ-CZ', 'FP1-F7', 'P3-O1', 'P4-O2', 'FP2-F8', 'P7-T7', 'F7-T7', 'FT9-FT10', 'C3-P3', 'C4-P4', 'FP2-F4'}


In [7]:
WINDOW_SIZE = 2 # sec
OVERLAP = 0 # sec
FREQUENCY_CHB = 256 #Hz
ictal_section_name = ['ictal', 'preictal_late', 'preictal_ontime', 'preictal_early', 'postictal', 'interictal']

origin = pd.read_csv("./patient_info_chb_split.csv")

patient_info_chb_segment = list()

for state in ictal_section_name:
    total_current_state = origin[origin['state']==state]

    # short-term data with overlap
    if state in ['ictal', 'preictal_late', 'preictal_ontime', 'preictal_early']:
        OVERLAP = 1

    # long-term data without overlap
    if state in ["interictal", "postictal"]:
        OVERLAP = 0

    for current_state in total_current_state.itertuples():
        filename, start, end = current_state[1], current_state[2], current_state[3]

        if end-start<WINDOW_SIZE:
            continue
        
        dirname = filename.split('_')[0]
        filepath = os.path.join('./data/CHB/',dirname,filename+'.edf')
        
        header = read_edf_header(filepath)
        startdate = header['startdate']
        
        start_from_0sec = int(start-startdate.timestamp())*FREQUENCY_CHB
        end_from_0sec = int(end-startdate.timestamp())*FREQUENCY_CHB            

        step_size = (WINDOW_SIZE-OVERLAP)*FREQUENCY_CHB #index
        window_size_with_frequency = int(WINDOW_SIZE*FREQUENCY_CHB)
        for window_start_index in range(start_from_0sec, end_from_0sec, step_size):
            # ["name", "start", "duration", "state", "frequency"]
            current_segment = [filename, window_start_index, window_size_with_frequency, state, FREQUENCY_CHB]
            patient_info_chb_segment.append(current_segment)

KeyboardInterrupt: 

In [ ]:
len(['Fp1-AVG', 'F3-AVG', 'C3-AVG', 'P3-AVG', 'Fp2-AVG', 'F4-AVG', 'C4-AVG', 'P4-AVG', 'F7-AVG', 'T1-AVG', 'T3-AVG', 'T5-AVG', 'O1-AVG', 'F8-AVG', 'T2-AVG', 'T4-AVG', 'T6-AVG', 'O2-AVG', 'Fz-AVG', 'Cz-AVG', 'Pz-AVG'])

21

In [ ]:
print(len(patient_info_chb_segment))

1770539


In [ ]:
df =pd.DataFrame(patient_info_chb_segment, columns=["name", "start", "duration", "state", "frequency"])

In [ ]:
df[df['state']=='ictal'].shape

(257581, 5)

In [ ]:
df1 = df.iloc[:df.shape[0]//2,:]
df2 = df.iloc[df.shape[0]//2:,:]

In [ ]:
df1.to_csv('./patient_info_chb_segment_1.csv', index=False)

In [ ]:
df2.to_csv('./patient_info_chb_segment_2.csv', index=False)

In [ ]:
from pyedflib import EdfReader
from pyedflib.highlevel import read_edf

In [ ]:
def read_segment(path, chn, start, n):
    """
    Returns the physical data of signal chn. When start and n is set, a subset is returned

    Parameters:	
    chn : int
    channel number

    start : int
    start pointer (default is 0)

    n : int
    length of data to read (default is None, by which the complete data of the channel are returned)

    digital: bool
    will return the signal in original digital values instead of physical values

    Examples
    ——–
    >>> import pyedflib
    >>> f = pyedflib.data.test_generator()
    >>> x = f.readSignal(0,0,1000)
    >>> int(x.shape[0])
    1000
    >>> x2 = f.readSignal(0)
    >>> int(x2.shape[0])
    120000
    >>> f.close()    
    """
    path = "./data/CHB/CHB001/CHB001_01.edf"
    f = EdfReader(path)
    segment = f.readSignal(chn,start,n,digital=True)
    

array([ 0.1953602 ,  0.1953602 ,  0.1953602 ,  0.1953602 ,  0.97680098,
        0.58608059, -1.75824176, -1.36752137,  2.93040293,  4.1025641 ])

In [ ]:
f.close()

array([[-1.45934066e+02,  1.95360195e-01,  1.95360195e-01, ...,
        -1.15262515e+01, -2.93040293e+00,  1.93406593e+01],
       [-1.04517705e+02,  1.95360195e-01,  1.95360195e-01, ...,
         2.36385836e+01,  2.75457875e+01,  3.06715507e+01],
       [-4.27838828e+01,  1.95360195e-01,  1.95360195e-01, ...,
         4.86446886e+01,  4.51282051e+01,  3.45787546e+01],
       ...,
       [-2.64713065e+02,  1.95360195e-01,  5.86080586e-01, ...,
         9.76800977e-01, -1.58241758e+01, -2.94993895e+01],
       [ 9.47496947e+01,  1.95360195e-01,  1.95360195e-01, ...,
        -7.22832723e+00, -1.03540904e+01, -1.34798535e+01],
       [ 4.47374847e+01,  1.95360195e-01,  1.95360195e-01, ...,
         1.69963370e+01,  2.24664225e+01,  2.63736264e+01]])